## Aprendizagem Automática 2023/2024

### 3rd Home assignment

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb


In [13]:
def present_reg_statistics(y_test, preds):
    print("The RVE is: ", explained_variance_score(y_test, preds))
    print("The rmse is: ", mean_squared_error(y_test, preds, squared=False))
    corr, pval=pearsonr(y_test, preds)
    print("The Correlation Score is is: %6.4f (p-value=%e)\n"%(corr,pval))
    print("The Maximum Error is is: ", max_error(y_test, preds))
    print("The Mean Absolute Error is: ", mean_absolute_error(y_test, preds))
    #plt.figure(figsize=(5,5))
    #plt.scatter(preds, y_test)
    #plt.plot((50, 350), (50, 350), c="r")
    #plt.grid()
    #plt.show()


### Load the data

In [3]:

X_train, X_ivs, y_train, col_names = pickle.load(open("drd2_data.pickle", "rb"))


X_train_df = pd.DataFrame(X_train, columns=col_names)#convert the arrays to pandas dataframes
X_ivs_df = pd.DataFrame(X_ivs, columns=col_names)


X_train_df
#how much the molecule inhibts the receptor=y

,D00,D01,D02,D03,D04,D05,D06,D07,D08,D09,...,FP2079,FP2080,FP2081,FP2082,FP2083,FP2084,FP2085,FP2086,FP2087,FP2088
0,541.280138,541.656,10.0,1.0,8.0,1.0,10.0,40.0,75.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,426.197714,426.582,5.0,1.0,9.0,1.0,4.0,30.0,60.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,348.183778,348.446,4.0,0.0,3.0,0.0,3.0,26.0,50.0,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1455.763803,1456.831,27.0,19.0,23.0,17.0,16.0,105.0,206.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,387.151368,387.886,4.0,0.0,4.0,0.0,4.0,27.0,50.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7332,467.149047,467.513,6.0,0.0,6.0,0.0,5.0,32.0,56.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7333,240.162649,240.350,2.0,0.0,3.0,0.0,2.0,18.0,38.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7334,510.317874,510.802,4.0,0.0,10.0,0.0,4.0,37.0,79.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7335,393.187483,393.556,4.0,2.0,5.0,1.0,5.0,28.0,55.0,5.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Data Preprocessing

In [4]:
#Scaling the data
scaler = StandardScaler()
Xs_train = pd.DataFrame(scaler.fit_transform(X_train_df), columns=X_train_df.columns)
Xs_ivs = pd.DataFrame(scaler.transform(X_ivs_df), columns=X_ivs_df.columns)

In [5]:
#Checking for missing values 
check_nan = Xs_train.isnull().values.any()
print(check_nan)

check_nan = Xs_ivs.isnull().values.any()
print(check_nan)


False
False


In [6]:
#Separate the dataset in training and testing

X_train, X_test, y_train, y_test = train_test_split(Xs_train, y_train, test_size=0.2, random_state=42)

### Feature Selection

### Random Forest for Feature Selection

In [9]:
N,M=X_train.shape
N,M
v=np.hstack((y_train.reshape((N,1)), X_train))
pd.DataFrame(v)


,0,1,2,3,4,5,6,7,8,9,...,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132
0,0.000000,0.130290,0.129406,0.858046,-0.081854,-0.088142,-0.058057,1.549653,0.116628,-0.045254,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
1,0.250000,-0.491862,-0.492954,-1.064899,-0.477479,-0.589590,-0.508809,-1.318794,-0.426772,-0.355960,...,-0.810066,-0.195463,-0.179087,-0.311512,5.928141,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
2,0.000000,-0.602119,-0.603136,-0.584163,-0.081854,-0.589590,-0.058057,-0.601682,-0.504401,-0.472474,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,5.029661,-0.323477,-0.195463
3,0.198970,-0.106337,-0.107602,0.377310,-0.477479,0.246156,-0.508809,0.832542,-0.038629,-0.122930,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,7.331694,-0.198821,-0.323477,-0.195463
4,0.507495,-0.079080,-0.079887,-0.103427,-0.477479,0.079007,-0.508809,0.115430,-0.116258,-0.200607,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5864,0.089334,-0.112316,-0.113142,0.136941,-0.081854,-0.589590,-0.058057,0.473986,-0.116258,-0.433636,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
5865,0.491644,0.278713,0.282670,-0.103427,-0.081854,-0.255292,-0.058057,-0.243126,0.194257,-0.045254,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,5.116060
5866,0.537492,-0.613018,-0.613986,-0.824531,-0.477479,-0.589590,-0.508809,-0.960238,-0.504401,-0.394798,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
5867,0.104399,0.014756,0.013408,-0.103427,-0.477479,-0.255292,-0.508809,-0.243126,0.038999,-0.084092,...,1.234467,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463


In [10]:
rfr=RandomForestRegressor(random_state=0)
sel = SelectFromModel(estimator=rfr, threshold='mean') #Change the threshold! See what happens!
sel.fit(X_train, y_train)

print("Importances: ", sel.estimator_.feature_importances_)

print("Default threshold: ", sel.threshold_)

features=sel.get_support()
Features_selected =np.arange(M)[features]
print("The features selected are columns: ", Features_selected)

Importances:  [3.90440067e-03 3.89625059e-03 2.62879089e-03 ... 8.90369571e-05
 3.95498492e-04 1.51153768e-04]
Default threshold:  0.00046904315196998124
The features selected are columns:  [   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   22   23   24   25   26   27   28   29
   30   31   32   33   34   35   36   37   38   39   40   41   42   50
   62   67   74   76   83   96   99  103  106  112  122  152  154  157
  165  172  175  180  186  198  199  200  214  216  229  232  234  243
  246  278  287  292  303  316  326  335  336  340  347  348  351  353
  357  362  364  370  371  377  381  386  390  396  421  440  442  444
  451  455  459  472  494  502  513  525  536  538  546  551  558  559
  561  565  566  569  570  572  608  615  617  623  630  640  646  675
  676  679  687  695  699  711  717  719  722  727  728  747  765  769
  770  778  790  808  811  815  819  824  839  851  853  867  871  873
  876  879  886  889  891  90

In [30]:
nX_train=sel.transform(X_train)
nX_test=sel.transform(X_test)
nX_ivs=sel.transform(Xs_ivs)

### KNN, SVM, DT, Ridge Regression, Random Forest 

### SVR

In [12]:

best_mse = float('inf')
best_params = {}

gammas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
Cs = [1, 10, 100, 1e3, 1e4, 1e5]

for gamma in gammas:
    for C in Cs:
        svr = SVR(gamma=gamma, C=C)
        svr.fit(nX_train, y_train)
        preds = svr.predict(nX_test)
        mse = mean_squared_error(y_test, preds)
        
        if mse < best_mse:
            best_mse = mse
            best_params = {'gamma': gamma, 'C': C}

best_svr = SVR(**best_params)
best_svr.fit(nX_train, y_train)
preds = best_svr.predict(nX_test)


In [14]:
print(best_svr)
present_reg_statistics(y_test, preds)

SVR(C=10, gamma=0.001)
The RVE is:  0.6269828205917798
The rmse is:  0.1683107548449815
The Correlation Score is is: 0.7919 (p-value=2.128546e-316)

The Maximum Error is is:  0.7813564151906202
The Mean Absolute Error is:  0.1269837317599825


### DT

In [15]:
best_mse = float('inf')
best_params = {}

# Define the range of hyperparameters to search
max_depths = [15, 20, 25, 30]  # Example depth values to search
min_samples_leafs = [15, 20, 25, 30]  # Example min samples leaf values

for max_depth in max_depths:
    for min_samples_leaf in min_samples_leafs:
        # Create and train the model with different hyperparameters
        dt = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        dt.fit(nX_train, y_train)
        preds = dt.predict(nX_test)
        mse = mean_squared_error(y_test, preds)
        
        # Update best parameters if this combination performs better
        if mse < best_mse:
            best_mse = mse
            best_params = {'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf}

# Train the model using the best hyperparameters
best_dt = DecisionTreeRegressor(**best_params)
best_dt.fit(nX_train, y_train)
preds = best_dt.predict(nX_test)



In [16]:
print(best_dt)
present_reg_statistics(y_test, preds)

DecisionTreeRegressor(max_depth=20, min_samples_leaf=15)
The RVE is:  0.40432994343713136
The rmse is:  0.2125402551096508
The Correlation Score is is: 0.6494 (p-value=1.513982e-176)

The Maximum Error is is:  0.8653546494666666
The Mean Absolute Error is:  0.16235852877975518


### Ensemble Models: Bagging-> KNN, DT

##### Decision Tree

In [18]:
from sklearn.model_selection import cross_val_score


best_score_bagging_dt = float('inf')
best_params_bagging_dt = {}

depths = [15, 20, 25]

for depth in depths:
    dt = DecisionTreeRegressor(max_depth=depth)
    bagging_dt = BaggingRegressor(base_estimator=dt, n_estimators=10, random_state=0)
    scores = cross_val_score(bagging_dt, nX_train, y_train, cv=5, scoring='explained_variance')
    mean_score = scores.mean()
    
    if mean_score > best_score_bagging_dt:
        best_score_bagging_dt = mean_score
        best_params_bagging_dt['max_depth'] = depth

#Train the model with the hyperparameters
dt = DecisionTreeRegressor(**best_params_bagging_dt)
bagging_dt = BaggingRegressor(base_estimator=dt, n_estimators=10, random_state=0)  
bagging_dt.fit(nX_train, y_train)
bagging_dt_preds = bagging_dt.predict(nX_test)

Bg_dt = present_reg_statistics (y_test, bagging_dt_preds)

The RVE is:  0.5730981168258943
The rmse is:  0.1799525250480272
The Correlation Score is is: 0.7571 (p-value=2.798392e-273)

The Maximum Error is is:  0.9230639854
The Mean Absolute Error is:  0.13400811925239725


##### KNN

In [22]:
best_score_bagging_knn = float('inf')
best_params_bagging_knn = {}

n_knn = [3,5,7]

for n_neighbors in n_knn:
    knn = KNeighborsRegressor(n_neighbors=n_neighbors)
    bagging_knn = BaggingRegressor(base_estimator=knn, n_estimators=10, random_state=0)
    scores = cross_val_score(bagging_knn, nX_train, y_train, cv=5, scoring='explained_variance')
    mean_score = scores.mean()
    
    if mean_score > best_score_bagging_knn:
        best_score_bagging_knn = mean_score
        best_params_bagging_knn['n_neighbors'] = n_neighbors


knn = KNeighborsRegressor(**best_params_bagging_knn)  
bagging_knn = BaggingRegressor(base_estimator=knn, n_estimators=10, random_state=0)
bagging_knn.fit(nX_train, y_train)
bagging_knn_preds = bagging_knn.predict(nX_test)

Bg_knn = present_reg_statistics (y_test, bagging_knn_preds)



The RVE is:  0.6165924916373591
The rmse is:  0.17054630684271474
The Correlation Score is is: 0.7857 (p-value=3.834948e-308)

The Maximum Error is is:  0.88132395182
The Mean Absolute Error is:  0.12536290686752044


### Adaboost: Decision Tree

In [28]:
best_score_adaboost_dt = float('inf')
best_params_adaboost_dt = {}

ab_depths = [10, 15, 20]

for depth in ab_depths:
    adaboost_dt = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=depth),
                                     n_estimators=50, learning_rate=1.0, random_state=0)
    scores = cross_val_score(adaboost_dt, nX_train, y_train, cv=5, scoring='explained_variance')
    mean_score = scores.mean()
    
    if mean_score > best_score_adaboost_dt:
        best_score_adaboost_dt = mean_score
        best_params_adaboost_dt['max_depth'] = depth


adaboost_dt = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(best_params_adaboost_dt),
                                n_estimators=50, learning_rate=1.0, random_state=0)
adaboost_dt = AdaBoostRegressor(**best_params_adaboost_dt)
adaboost_dt.fit(nX_train, y_train)
adaboost_dt_preds = adaboost_dt.predict(nX_test)

AB_dt =  present_reg_statistics (y_test, adaboost_dt_preds )
AB_dt

/Users/martaaraujo/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass criterion={} as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


The RVE is:  0.19502851169710167
The rmse is:  0.24841768534858083
The Correlation Score is is: 0.4994 (p-value=1.900603e-93)

The Maximum Error is is:  0.7506088518117645
The Mean Absolute Error is:  0.20449309039214755


### Gradient Boosting 

In [25]:
best_score_grad_boost = -float('inf')  # Initialize to a very low value
best_params_grad_boost = {}

for depth in [3, 5, 7]:
    grad_boost = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=depth, random_state=0)
    scores = cross_val_score(grad_boost, nX_train, y_train, cv=5, scoring='explained_variance')
    mean_score = scores.mean()
    print(f"Depth: {depth}, Mean Score: {mean_score}")  # Print mean scores for debugging
    
    if mean_score > best_score_grad_boost:
        best_score_grad_boost = mean_score
        best_params_grad_boost['max_depth'] = depth

if 'max_depth' in best_params_grad_boost:  # Check if the key exists before accessing
    m_d = best_params_grad_boost['max_depth']
    grad_boost = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=m_d, random_state=0)
    grad_boost.fit(nX_train, y_train)
    grad_boost_preds = grad_boost.predict(nX_test)

    GB = present_reg_statistics(y_test, grad_boost_preds)
    print(GB)  # Check the output statistics
else:
    print("No best max_depth found.")


Depth: 3, Mean Score: 0.4884680416536519
Depth: 5, Mean Score: 0.5825294344259866
Depth: 7, Mean Score: 0.6217275762282396
The RVE is:  0.6119852984723875
The rmse is:  0.17155141516519923
The Correlation Score is is: 0.7836 (p-value=2.307629e-305)

The Maximum Error is is:  0.8428387061781568
The Mean Absolute Error is:  0.12884387293280794
None


### XGBoost

In [26]:
best_score_xgboost = -float('inf')
best_params_xgboost = {}

for depth in [3, 5, 7]:
    xgboost = xgb.XGBRegressor(n_estimators=100, max_depth=depth, learning_rate=0.1, random_state=0)
    scores = cross_val_score(xgboost, nX_train, y_train, cv=5, scoring='explained_variance')
    mean_score = scores.mean()
    
    if mean_score > best_score_xgboost:
        best_score_xgboost = mean_score
        best_params_xgboost['max_depth'] = depth

xgb_md = best_params_xgboost['max_depth']
xgboost = xgb.XGBRegressor(n_estimators=100, max_depth=xgb_md, learning_rate=0.1, random_state=0)
xgboost.fit(nX_train, y_train)
xgboost_preds = xgboost.predict(nX_test)

XGB = present_reg_statistics (y_test, xgboost_preds )
XGB

The RVE is:  0.6089058039476718
The rmse is:  0.17220827530630956
The Correlation Score is is: 0.7823 (p-value=1.202609e-303)

The Maximum Error is is:  0.8528495877981186
The Mean Absolute Error is:  0.1292475587533101


### Estimating y_ivs

In [31]:
#our best model was SVR.
svr = SVR(gamma=0.001, C=10)
svr.fit(nX_train, y_train)
preds = svr.predict(nX_ivs)
print('y_ivs:',preds)



y_ivs: [ 3.21618645e-01  5.67140348e-01  3.56575039e-01  3.87816361e-01
  1.47783414e-01 -2.32611729e-02  3.88704028e-01  7.86337611e-01
  2.88423028e-01  6.72789212e-01  5.36831810e-01  5.00713145e-01
  5.58895433e-01  1.26198066e-01  2.47152893e-01  1.98738706e-01
  2.10015056e-01  3.31103346e-01  4.02695744e-01  2.59648152e-02
  8.91738337e-01  2.54842884e-01  5.26635598e-01  1.94763568e-01
  1.85327859e-01  3.53022095e-01  1.65073351e-01  1.82655778e-01
  2.05809622e-01  2.22439422e-01 -1.06726186e-02  4.33018900e-01
  3.22136869e-01  5.07384644e-01  7.88432011e-01  3.85918002e-01
  1.96574107e-01  5.83590390e-01  2.91419852e-01  5.36953006e-01
  2.14407813e-01 -1.82242903e-02  2.07291192e-01  4.42951752e-01
  3.07263130e-01  4.22397557e-01  7.97837974e-01 -9.90442577e-02
  3.91384406e-01  3.54191403e-01  3.00872337e-01  1.25103436e-01
  4.62608968e-02  1.98279345e-01  2.95968778e-01  7.64896150e-01
  4.34961663e-01  2.53218602e-01  4.82787959e-01  6.14596821e-01
  2.88817098e-01  